# Flyer ML Optimization (Rebuild) — Multi‑Task GP with Ax/BoTorch

This notebook rebuilds the Bayesian Optimization pipeline so the surrogate **leverages RPM and vibration as *measured side metrics*** (not optimizable inputs). We jointly model the objective, RPM, and vibration with a **Multi‑Task GP** so information can transfer across tasks and handle **partial observations**. Acquisition is optimized **only for the objective task**.

## 0) Setup
Uncomment installs if running in a fresh Colab environment.

In [ ]:
# If needed on Colab:
# !pip -q install ax-platform botorch gpytorch torch torchvision torchaudio
import os
from typing import List, Dict
import numpy as np
import pandas as pd
import torch

from botorch.models.multitask import MultiTaskGP
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.acquisition import qExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

torch.set_default_dtype(torch.double)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1) Configuration
👉 Only provide bounds for **design parameters** (variables the BO loop can suggest). You do **not** need bounds for RPM/vibration.

In [ ]:
# ==== USER CONFIGURATION ====
# Path to your CSV (or leave None and provide/construct df manually in next cell):
DATA_CSV = None  # e.g., '/content/flyer_trials.csv'

# Objective column to optimize (e.g., 'ld_ratio', 'lift', 'efficiency'):
OBJECTIVE_COL = 'ld_ratio'

# Side metrics available in your dataset (observed, not optimized):
SIDE_TASKS = ['rpm', 'vibration']  # drop any that don't exist in your data

# Column for vibration (should be 0/1 after mapping if needed):
VIBRATION_COL = 'vibration'

# Design parameters (optimizable inputs):
DESIGN_VARS = [
    # e.g., 'symmetry', 'pitch', 'chord', 'radius', 'twist'
]

# Bounds for each design var (required):
BOUNDS: Dict[str, tuple] = {
    # 'pitch': (10.0, 40.0),
    # 'chord': (1.0, 3.0),
}

# Number of suggestions to propose:
N_CANDIDATES = 5

# Standardize outcomes per task:
STANDARDIZE_Y = True

# ==== END CONFIGURATION ====
assert len(DESIGN_VARS) > 0, "Please set DESIGN_VARS to your design parameter columns."
assert set(DESIGN_VARS).issubset(set(BOUNDS.keys())), "Bounds must be provided for all DESIGN_VARS."

## 2) Load & Preprocess Data
Expected columns: `DESIGN_VARS + [OBJECTIVE_COL] + SIDE_TASKS`. RPM numeric; vibration 0/1. Missing values are OK (handled per-task).

In [ ]:
if DATA_CSV is not None and os.path.exists(DATA_CSV):
    df = pd.read_csv(DATA_CSV)
    print(f"Loaded: {DATA_CSV}, shape={df.shape}")
else:
    # Synthetic fallback so the notebook runs without your CSV
    rng = np.random.default_rng(0)
    if not DESIGN_VARS:
        DESIGN_VARS.extend(['pitch', 'chord'])
    if not BOUNDS:
        BOUNDS.update({'pitch': (10.0, 40.0), 'chord': (1.0, 3.0)})
    n = 100
    pitch = rng.uniform(BOUNDS['pitch'][0], BOUNDS['pitch'][1], size=n)
    chord = rng.uniform(BOUNDS['chord'][0], BOUNDS['chord'][1], size=n)
    ld = (pitch - 25.0)**2 * -0.01 + (chord - 2.0)**2 * -0.2 + 10 + rng.normal(0, 0.2, size=n)
    rpm = 3000 + 80*(pitch-25) + rng.normal(0, 50, size=n)
    vib = (rng.random(size=n) < (0.2 + 0.1*np.clip(chord-1.0, 2.0, None))).astype(int)
    df = pd.DataFrame({'pitch': pitch, 'chord': chord, 'ld_ratio': ld, 'rpm': rpm, 'vibration': vib})
    print("Created synthetic dataset for demonstration.")
    display(df.head())

# Ensure binary vibration if present
if VIBRATION_COL in df.columns:
    if not set(pd.Series(df[VIBRATION_COL]).dropna().unique()).issubset({0,1}):
        mapping = {True:1, False:0, 'yes':1, 'no':0, 'high':1, 'low':0, 'y':1, 'n':0}
        df[VIBRATION_COL] = df[VIBRATION_COL].map(lambda x: mapping.get(x, x)).astype(float)

# Activate only tasks present in the data
SIDE_TASKS = [t for t in SIDE_TASKS if t in df.columns]
TASK_LIST = [OBJECTIVE_COL] + SIDE_TASKS
print("Active tasks:", TASK_LIST)

# Tensor bounds for BoTorch
bounds = torch.tensor([[BOUNDS[k][0] for k in DESIGN_VARS], [BOUNDS[k][1] for k in DESIGN_VARS]], dtype=torch.double, device=device)

## 3) Build Multi‑Task Training Tensors
We stack rows per task with a **task feature** as the last column of X.

In [ ]:
task_to_index = {t:i for i,t in enumerate(TASK_LIST)}
print("Task indices:", task_to_index)

def build_multitask_tensors(df: pd.DataFrame, design_vars: List[str], task_cols: List[str]):
    Xs, Ys = [], []
    for t in task_cols:
        sub = df[design_vars + [t]].dropna()
        if sub.empty:
            continue
        X = torch.tensor(sub[design_vars].to_numpy(), dtype=torch.double)
        tfeat = torch.full((X.shape[0], 1), float(task_to_index[t]), dtype=torch.double)
        X_mt = torch.cat([X, tfeat], dim=1)
        y = torch.tensor(sub[t].to_numpy(), dtype=torch.double).unsqueeze(-1)
        Xs.append(X_mt)
        Ys.append(y)
    X_all = torch.cat(Xs, dim=0).to(device)
    Y_all = torch.cat(Ys, dim=0).to(device)
    return X_all, Y_all

X_all, Y_all = build_multitask_tensors(df, DESIGN_VARS, TASK_LIST)
print("X_all:", X_all.shape, "Y_all:", Y_all.shape)

## 4) Fit a Multi‑Task GP (ICM)
`MultiTaskGP` learns a shared input kernel + a task covariance (coregionalization).

In [ ]:
input_dim = len(DESIGN_VARS)
task_feature = input_dim
outcome_tf = Standardize(m=1) if STANDARDIZE_Y else None

mtgp = MultiTaskGP(
    train_X=X_all,
    train_Y=Y_all,
    task_feature=task_feature,
    outcome_transform=outcome_tf,
).to(device)
mll = ExactMarginalLogLikelihood(mtgp.likelihood, mtgp)
fit_gpytorch_model(mll)
mtgp.eval()
print("Fitted MultiTaskGP over:", TASK_LIST)

## 5) Acquisition on Objective Task Only
We wrap the model to **append the objective task index** internally, then optimize qEI in the design space.

In [ ]:
from botorch.models.model import Model

def propose_candidates(model: Model, bounds: torch.Tensor, n: int = 5, raw_samples: int = 256, q: int = 1):
    obj_mask = (X_all[:, -1] == 0)
    Y_obj = Y_all[obj_mask]
    best_f = Y_obj.max()

    class ObjectiveTaskModel(torch.nn.Module):
        def __init__(self, base_model, task_index: float, d: int):
            super().__init__()
            self.base_model = base_model
            self.task_index = torch.tensor(task_index, dtype=torch.double, device=Y_obj.device)
            self.d = d
        def posterior(self, X, output_indices=None, observation_noise=False, **kwargs):
            if X.dim() == 2:
                tcol = self.task_index.expand(X.shape[0], 1)
                X_full = torch.cat([X, tcol], dim=1)
            elif X.dim() == 3:
                b, q, d = X.shape
                tcol = self.task_index.expand(b*q, 1)
                X2 = X.view(b*q, d)
                X_full = torch.cat([X2, tcol], dim=1).view(b, q, d+1)
            else:
                raise RuntimeError("Unexpected X shape")
            return self.base_model.posterior(X_full, output_indices=output_indices, observation_noise=observation_noise, **kwargs)

    wrapped = ObjectiveTaskModel(model, task_index=0.0, d=len(DESIGN_VARS))
    acqf = qExpectedImprovement(model=wrapped, best_f=best_f)

    cand_list = []
    for _ in range(n):
        cand, _ = optimize_acqf(
            acq_function=acqf,
            bounds=bounds,
            q=q,
            num_restarts=10,
            raw_samples=raw_samples,
        )
        cand_list.append(cand.detach().cpu().numpy())
    C = np.vstack(cand_list)
    return pd.DataFrame(C, columns=DESIGN_VARS)

candidates_df = propose_candidates(mtgp, bounds, n=N_CANDIDATES)
candidates_df

## 6) Save Recommendations
Export proposed design points to CSV.

In [ ]:
out_csv = 'next_experiments_multitask_gp.csv'
candidates_df.to_csv(out_csv, index=False)
out_csv

## 7) (Optional) Single‑Task Baseline
Fit a standard Single‑Task GP on the objective only (ignores RPM/vibration).

In [ ]:
obj_mask = (X_all[:, -1] == 0)
X_obj = X_all[obj_mask][:, :len(DESIGN_VARS)]
Y_obj = Y_all[obj_mask]

stgp = SingleTaskGP(train_X=X_obj, train_Y=Y_obj, outcome_transform=Standardize(m=1) if STANDARDIZE_Y else None).to(device)
mll2 = ExactMarginalLogLikelihood(stgp.likelihood, stgp)
fit_gpytorch_model(mll2)
stgp.eval()

best_f = Y_obj.max()
acq_st = qExpectedImprovement(model=stgp, best_f=best_f)
cand_st, _ = optimize_acqf(
    acq_function=acq_st,
    bounds=bounds,
    q=1,
    num_restarts=10,
    raw_samples=256,
)
baseline_df = pd.DataFrame(cand_st.detach().cpu().numpy(), columns=DESIGN_VARS)
baseline_out = 'next_experiments_single_task_gp.csv'
baseline_df.to_csv(baseline_out, index=False)
baseline_out

## Notes
- You **do not** provide bounds for RPM/vibration (they are *observed outputs*, not inputs).
- Partial observations are fine: MTGP learns task correlations and shares strength.
- If vibration is string/categorical, map to 0/1 before training.
- To integrate with Ax, wrap this fitted BoTorch model in a custom Ax `BotorchModel` and fix the task feature to 0 in evaluation.